In [63]:
import requests
import re
from typing import Dict
import time
import logging
import json
import os
from datetime import datetime
import pickle
from typing import Dict, List

import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

tqdm.pandas()

In [64]:
load_dotenv()

INFERENCE_MODEL = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
DEEPINFRA_TOKEN = os.environ.get('DEEPINFRA_API_TOKEN')

In [65]:
rools = '\n'.join(map(str, json.load(open('data/resume_rools.json'))))

In [66]:
prompt = """
Ты — ИИ-модератор резюме. Проверь текст резюме на соответствие следующим правилам. Если есть нарушения, верни JSON-объект с типом нарушения и фрагментом текста, который нарушает правило. Если нарушений нет, верни статус "OK".

**Правила:** 
{rools}

**Инструкции:**

- Тщательно проанализируй каждое предложение в резюме.
- Если нарушений нет, верни "status": "OK" и пустой массив violated_rules.
- Если фрагмент нарушает правило, укажи его точную цитату в resume_fragment.
- Ответ должен содержать твои рассуждения по каждому правилу оканчивающийся вердиктом, затем должен быть результат в формате JSON, без Markdown и комментариев.

Формат ответа: Рассуждения:, Результат: { "status": "OK" | "violation", "violated_rules": [] | [ { "id": "rule_id", "condition": "Текст условия правила на русском", "resume_fragment": "Точная цитата из резюме" } ] }

**Текст резюме:**
{resume_text}
"""

In [67]:
print(prompt.replace('{rools}', rools))


Ты — ИИ-модератор резюме. Проверь текст резюме на соответствие следующим правилам. Если есть нарушения, верни JSON-объект с типом нарушения и фрагментом текста, который нарушает правило. Если нарушений нет, верни статус "OK".

**Правила:** 
{'id': 'rule_1', 'condition': 'Резюме должно содержать информацию о трудовой деятельности если желаемая должность требует опыта'}
{'id': 'rule_2', 'condition': 'Резюме должно содержать профессию/должность или должности, на которые претендует соискатель, должность должжна быть реальной профессией'}
{'id': 'rule_4', 'condition': 'Резюме должно быть заполнено на русском языке'}
{'id': 'rule_5', 'condition': 'Информация о работодателе не должна содержать сведений, не относящихся к работодателю, если работодатель ИП это не нарушение правил'}
{'id': 'rule_6', 'condition': 'Информация о резюме не должна содержать сведений, не относящихся к соискателю'}
{'id': 'rule_7', 'condition': 'Информация не должна содержать слова и выражения, не соответствующие норм

In [96]:
def parse_answer(response_str: str) -> Dict[str, str]:
    parts = response_str.split("Результат:")
    
    reasoning = parts[0].replace("Рассуждения:\n", "").strip()

    try:
        result_dict = json.loads(parts[1])
    except json.JSONDecodeError:
        result_dict = {"error": "Invalid JSON format"}
    
    return {
        "reasoning": reasoning,
        "result": result_dict
    }

In [97]:
def get_answer(resume_text: str) -> Dict[str, str]:
    url = "https://api.deepinfra.com/v1/openai/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {DEEPINFRA_TOKEN}"
    }
    
    formatted_prompt = prompt.replace('{rools}', rools).replace('{resume_text}', resume_text)
    
    data = {
        "model": INFERENCE_MODEL,
        "messages": [
            {
                "role": "user",
                "content": formatted_prompt
            }
        ],
        "temperature": 0,
        "seed": 42
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    
    answer_content = response.json()['choices'][0]['message']['content']
    
    return parse_answer(answer_content)

In [98]:
df = pd.read_csv('./data/processed_resume.csv')

In [ ]:
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('processing.log'),
        logging.StreamHandler()
    ]
)

def process_dataframe(df, checkpoint_interval=20):
    last_checkpoint = get_last_checkpoint()
    start_index = 0
    results = []
    
    if last_checkpoint:
        start_index = last_checkpoint['index'] + 1
        results = last_checkpoint['results']
        logging.info(f"Resuming from index {start_index}")

    try:
        for i in tqdm(range(start_index, len(df)), initial=start_index, total=len(df)):
            time.sleep(5)
            try:
                result = get_answer(df.text_data[i])
                results.append(result)
                
                if (i - start_index) % checkpoint_interval == 0:
                    save_checkpoint(i, results)
                    
            except Exception as e:
                logging.error(f"Error processing row {i}: {str(e)}")
                results.append({'error': str(e)})
                
        save_checkpoint(len(df)-1, results)
        return pd.DataFrame(results)
        
    except KeyboardInterrupt:
        logging.info("Process interrupted. Saving last checkpoint...")
        save_checkpoint(i-1, results)
        return pd.DataFrame(results)

def save_checkpoint(index, results):
    checkpoint = {
        'timestamp': datetime.now(),
        'index': index,
        'results': results
    }
    
    with open('checkpoint.pkl', 'wb') as f:
        pickle.dump(checkpoint, f)
        
    logging.info(f"Checkpoint saved at index {index}")

def get_last_checkpoint():
    if os.path.exists('checkpoint.pkl'):
        try:
            with open('checkpoint.pkl', 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            logging.error(f"Error loading checkpoint: {str(e)}")


result_df = process_dataframe(df)
result_df.to_csv('final_results.csv', index=False)

2025-02-24 13:13:27,188 - INFO - Checkpoint saved at index 0
2025-02-24 13:23:42,580 - INFO - Checkpoint saved at index 20
2025-02-24 13:39:37,605 - INFO - Checkpoint saved at index 40
2025-02-24 13:51:23,843 - INFO - Checkpoint saved at index 60
2025-02-24 14:08:39,584 - INFO - Checkpoint saved at index 80
2025-02-24 14:21:17,773 - INFO - Checkpoint saved at index 100
2025-02-24 14:34:46,062 - INFO - Checkpoint saved at index 120
2025-02-24 14:45:10,678 - INFO - Checkpoint saved at index 140
2025-02-24 14:58:44,553 - INFO - Checkpoint saved at index 160
2025-02-24 15:08:07,653 - INFO - Checkpoint saved at index 180
2025-02-24 15:18:31,278 - INFO - Checkpoint saved at index 200
2025-02-24 15:31:56,468 - INFO - Checkpoint saved at index 220
 26%|██▌       | 234/905 [2:28:26<4:16:10, 22.91s/it]  

In [128]:
df['llama_answer'] = result_df['result']

In [130]:
df['llama_status'] = result_df['result'].apply(lambda el: el.get('status'))

In [132]:
df['llama_reasoning'] = result_df['reasoning']

In [134]:
df['llama_status'].value_counts()

llama_status
OK           602
violation    300
Name: count, dtype: int64

In [135]:
df.loc[df['Результат'] == 'Отклонено', 'llama_status']

198    violation
242    violation
425           OK
428    violation
822    violation
Name: llama_status, dtype: object

In [136]:
df.loc[df['Результат'] == 'Принято', 'llama_status'].value_counts()

llama_status
OK           497
violation    249
Name: count, dtype: int64

In [142]:
false_negative = (df.loc[df['Результат'] == 'Отклонено', 'llama_status'] == 'OK').astype(int).sum()
false_positive = (df.loc[df['Результат'] == 'Принято', 'llama_status'] == 'violation').astype(int).sum()
recall = (df.loc[df['Результат'] == 'Отклонено', 'llama_status'] == 'violation').astype(int).mean()
precision = (df.loc[df['Результат'] == 'Принято', 'llama_status'] == 'OK').astype(int).mean()
f1_score = 2 * (recall * precision) / (recall + precision)

In [143]:
print(f'false negative: {false_negative}')
print(f'false positive: {false_positive}')
print(f'recall: {recall:.2f}')
print(f'precision: {precision:.2f}')
print(f'f1-score: {f1_score:.2f}')

false negative: 1
false positive: 249
recall: 0.80
precision: 0.66
f1-score: 0.73


In [146]:
df.to_csv('data/processed_resume.csv')